In [14]:
import pandas as pd
import re
import numpy as np
import gzip
import concurrent.futures

from collections import defaultdict
from urllib.request import urlopen
'''
To run extraction on your computer you first need to download the files from:
https://hgdownload.soe.ucsc.edu/goldenPath/hg38/multiz100way/.

Run the commaמds:
mut_extractor = MPMutExtractor(folder, file_names, species, url)
mut_extractor.get_mutation_counts()

When the parameters are:
folder: The folder containing the downloaded maf files with the different chromosomes alignment data
file_names: The name of all the files with the data from the folder
species: A String array with the three (scientific) names of the species you want to analyse.
example = ["Ovis aries", "Capra hircus", "Bos taurus"], where Bos taurus is the outgroup
url - no need to specify, but should contain the url of the multiz100way in UCSC
'''



# Multiprocessing mutational count extractor
class MPMutExtractor:
    """
    This class uses multiprocessing to extract mutation counts
    It extracts counts from multiple chromosome files concurrently using Chromosome_MutSig class.
    
    Important Attributes:
    species_names (String array): scientific name of the 3 species to be used (species[2]) is the outlgroup.
    folder (String): The path of the folder containing alignment files
    file_names (String array): The file names that the sequencing files are in.
    """
    

    def __init__(self, 
                 folder = r"C:\Users\KerenYlab\Downloads" + "\\", 
                 file_names = ['chr1.maf.gz'],
                 species = ["Ovis aries", "Capra hircus", "Bos taurus"],
                 url = "https://hgdownload.soe.ucsc.edu/goldenPath/hg38/multiz100way/"):
        
        """
        A constructor to initialize a new MPMutExtractor instance.
        
        Parameters:
        self.html : html page information with about maf alignment files
        folder, file_names, species_names : as described above
        """
        
        
        page = urlopen(url)
        html_bytes = page.read()
        self.html = html_bytes.decode("utf-8")
        self.folder = folder
        self.file_names = file_names
        self.species_names = species
    
    
    def get_mutation_counts(self):
        """
        This function that runs the concurrent mutation count.

        Returns:
        a list of the mutation count for each chromosome (each mutation count is a dict)
        """
        
        self.__get_species_alignment_names(self.species_names)     
        results = None
        
        # Create an Executor object that runs the processes
        with concurrent.futures.ThreadPoolExecutor() as executor:
            results = executor.map(self.__get_chromosome_mutational_count, self.file_names)
            self.mutation_counts = list(results)
        return self.mutation_counts
            
        
    # The function that runs the mutation count on a single chromosome
    # Returns the mutation count from the chromosome (a dict)
    def __get_chromosome_mutational_count(self, file_name):
        chromosome_mutext = Chromosome_MutExt(self.folder, file_name, self.chosen_species)
        mutational_counts = chromosome_mutext.get_mutation_counts()
        return mutational_counts
    
        
    # finds common names
    def __find_common_names(self):
        start = re.search("Assemblies used in these alignments:", self.html)
        end = re.search("---------------------------------------------------------------", self.html)
        text = self.html[start.end():end.start()]

        names = []
        for line in text.splitlines():
            if (line != '') and not line.isspace() and line[0] != '=':
                names.append(line[25:56].strip(' '))
        common_names = '\n'.join(names)
        self.common_names = common_names
        return common_names


    #finds names for alignment
    def __find_alignment_names(self):
        names = []
        start = re.search("Assemblies used in these alignments:", self.html)
        end = re.search("---------------------------------------------------------------", self.html)
        text = self.html[start.end():end.start()]
        for line in text.splitlines():
            if (line == '') or line.isspace() or line[0] == '=':
                continue
            names.append(re.findall("/\S*\s", line)[-1][1:-1])
        alignment_names = '\n'.join(names)
        self.alignment_names = alignment_names
        return alignment_names
    
    #gets the alignment names for the 3 species given to it
    def __get_species_alignment_names(self, species = ["Ovis aries", "Capra hircus", "Bos taurus"]):
        
        if not hasattr(self, "common_names") or not hasattr(self, "alignment_names"):
            self.common_names = self.__find_common_names()
            self.alignment_names = self.__find_alignment_names()

        common_names = self.common_names.split('\n')
        alignment_names = self.alignment_names.split('\n')
        
        chosen_species = [None]*3
        
        # 3 chosen taxa = Ovis aries, Capra hircus and outgroup Bos taurus
        chosen_species[0] = np.array(alignment_names)[(np.array(common_names) == species[0])][0]
        chosen_species[1] = np.array(alignment_names)[(np.array(common_names) == species[1])][0]
        chosen_species[2] = np.array(alignment_names)[(np.array(common_names) == species[2])][0]
        self.chosen_species = chosen_species
        return chosen_species
    
    

#sigle chromosome mutation counts extractor
class Chromosome_MutExt: 
    """
    This class extracts in practice the mutation counts from a given chromosome
    
    Important Attributes:
    chosen_species (String array): scientific name of the 3 species to be used (species[2]) is the outgroup.
    folder (String): The path of the folder containing alignment file
    file_name (String): The file name that the sequencing data is in.
    mutation_count(dict): contains the mutation count extracted from the file
    """
    def __init__(self,
                 folder = r"C:\Users\KerenYlab\Downloads" + "\\", 
                 file_name = 'chr1.maf.gz',
                 species = ['oviAri3', 'capHir1', 'bosTau8']):
        
        
        self.chosen_species = species
        self.file_name = file_name
        self.folder = folder
        self.species1_mutation_count = defaultdict(int)
        self.species2_mutation_count = defaultdict(int)
        
        
    def get_mutation_counts(self):
        
        '''
        Parses the file line by line to avoid reading too much into memory
        Divides the lines into chunks (according to alignment file structure),
        And processes each chunk using __parse_paragraph

        Returns:
            The final count from the whole file
        '''
        
        species = ["s " + s for s in self.chosen_species]
        self.species1_mutation_count = defaultdict(int)
        self.species2_mutation_count = defaultdict(int)

        with gzip.open(self.folder + self.file_name, 'rt', encoding='utf-8') as file:
            chunk = [file.readline()]
            for line in file:
                if line[0] != 'a': 
                    chunk.append(line)
                else: # beggining of new chunk
                    chunk = ''.join(chunk)
                    if species[0] in chunk and species[1] in chunk and species[2] in chunk:
                        self.__parse_paragraph(chunk, species)
                    chunk = [line]
        # merges the 192 categories to 96
        self.species1_final_mutation_count = self.__get_final_count(self.species1_mutation_count)
        self.species2_final_mutation_count = self.__get_final_count(self.species2_mutation_count)
        return (self.species1_final_mutation_count, self.species2_final_mutation_count)
                       
                    
    anti_pattern = r"[^ATGC]"
    name_regex = re.compile(r"s \w*.")
    sequence_regex = re.compile(r"[atgcnATGCN-]+$")
    
    def __parse_paragraph(self, paragraph, species):
        '''
        Parses a single chunk containing sequencing data from the 3 species (and others).
        Extracts from it the mutation count and adds it to mutation_count dict
        '''
        sequences = {}
        for line in paragraph.splitlines():
            species_name = self.name_regex.findall(line)
            if species_name and species_name[0][:-1] in species:
                sequence = self.sequence_regex.findall(line)
                if sequence:
                    sequences[species_name[0][:-1]] = sequence[0].upper()
                else:
                    print("failed:")
                    print(line)
                    break;
        return self.__count_mutations(sequences, species)

    

    def __count_mutations(self, sequences, species):
        '''
        Counts from the 3 sequences all the mutations that can be detected
        Counts only mutations between the two sister taxas, in which the outlies has
        a nucleotide that is similar to that of one of the taxas.
        Adds the counts to mutation_count
        '''
        
        species1_mutation_count = self.species1_mutation_count
        species2_mutation_count = self.species2_mutation_count
        df = pd.DataFrame(columns = species)
        df[species[0]] = list(sequences[species[0]])
        df[species[1]] = list(sequences[species[1]])
        df[species[2]] = list(sequences[species[2]])
        df[species[0] + " neighbors"] = self.__get_neighbors(sequences[species[0]])
        df[species[1] + " neighbors"] = self.__get_neighbors(sequences[species[1]])
        df[species[2] + " neighbors"] = self.__get_neighbors(sequences[species[2]])
        # creates a boolean column of all point mutation between sister taxa (the nucleotide is different and is not a gap)
        # includes only mutations with adjacent nucleotides for all 3 taxa
        df["mutations"] = ((df[species[0]] != df[species[1]]) & 
                           (df[species[0] + " neighbors"] != '-') & 
                           (df[species[1] + " neighbors"] != '-') & 
                           (df[species[2] + " neighbors"] != '-'))

        # creates a boolean column with only the mutation that happend in one taxa and not in the other
        # this is important for establishing the type of mutation that occurred
        # df[species[0] + " neighbors"].iloc[df["mutations"] & ((df[species[0]] == df[species[2]])]

        mutations = df.loc[df["mutations"] & (df[species[0]] == df[species[2]])]
        for mut in mutations.index:
            mutation = mutations.loc[mut][species[1] + " neighbors"]
            mutation = mutation[0] + mutations.loc[mut][species[2]] + mutation[2] + "->" + mutation[1]
            species1_mutation_count[mutation] += 1

        mutations = df.loc[df["mutations"] & (df[species[1]] == df[species[2]])]
        for mut in mutations.index:
            mutation = mutations.loc[mut][species[0] + " neighbors"]
            mutation = mutation[0] + mutations.loc[mut][species[2]] + mutation[2] + "->" + mutation[1]
            species2_mutation_count[mutation] += 1

        self.species1_mutation_count = species1_mutation_count
        self.species2_mutation_count = species2_mutation_count
        return df

    #Calculated for each position the it's neighboring nucleotides (ignores '-' in alignment)
    def __get_neighbors(self, string):
        nucleotides = {'A','T','G','C'}
        sequence = re.sub(self.anti_pattern, '', string)
        sequence_len = len(sequence)
        i = 0
        neighbors = []
        for idx, char in enumerate(string):
            if i == sequence_len-1 or i == 0:  
                neighbors.append('-')
                if char in nucleotides:
                    i+=1
            elif char not in nucleotides:
                neighbors.append('-')
            else:
                neighbors.append(sequence[i-1:i+2])
                i += 1
        return neighbors

    
    complement = {'A':'T',
                  'T':'A',
                  'C':'G',
                  'G':'C'}
    
    # merges the mutations properly to reach 96 mutational categories
    def __get_complement(self, mutation):
        comp_mutation = ''.join([self.complement[nuc] for nuc in mutation[2::-1]])
        comp_mutation += "->" + self.complement[mutation[-1]]
        return comp_mutation

    def __get_final_count(self, mutation_count):
        merged_categories = defaultdict(int)

        for mutation, count in mutation_count.items():
            if mutation[1] in {'A', 'C'}:
                merged_categories[self.__get_complement(mutation)] += count
            else:
                merged_categories[mutation] += count
        return merged_categories


In [13]:
cm = Chromosome_MutExt()
cm.get_mutation_counts()

91


92